### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
from datetime import datetime 
import matplotlib.pyplot as plt
import scipy.stats as st
import numpy as np
# Read Ironore_price_market_index_test.csv File and store into Pandas data frame
commodity_to_load = "Resources/Ironore_price_market_index_test.csv"
purchase_data = pd.read_csv(commodity_to_load)
#purchase_data

In [2]:
#remove empty columns, fix naming error and bring Date in line with other dataframes
clean_data = purchase_data[['Date','Iron Price','Silver Price','Gold Price',' Copper Price','S&P GSCI Price']]
clean_data = clean_data.rename(columns={' Copper Price': 'Copper_Price',
                                       'Iron Price':'Iron_Price',
                                        'Silver Price':'Silver_Price',
                                        'Gold Price':'Gold_Price',
                                       'S&P GSCI Price':'S&P_GSCI_Price'})

clean_data['Date'] = clean_data['Date'].str[2:]
clean_data['Date'] = pd.to_datetime(clean_data['Date']).dt.to_period('M')
clean_data

#save data to Commodities_clean.csv
clean_data.to_csv('Outputs/Commodities_clean.csv')

In [3]:
iron_data = clean_data[['Date','Iron_Price']]


In [4]:
#https://www.kaggle.com/docs/api
#https://stackoverflow.com/questions/64393520/r-not-working-as-lineterminator-within-python-csv-writer
# Read all_perth_310121.csv File and store into Pandas data frame
perth_to_load = "Resources/all_perth_310121.csv"

perth_data = pd.read_csv(perth_to_load)


In [5]:
#fix up Date_sold format errors and bring in line with other datasets
perth_data["DATE_SOLD"] = perth_data["DATE_SOLD"].str.replace('\r','')
perth_data["DATE_SOLD"] = pd.to_datetime(perth_data["DATE_SOLD"]).dt.to_period('M')
perth_data

,ADDRESS,SUBURB,PRICE,BEDROOMS,BATHROOMS,GARAGE,LAND_AREA,FLOOR_AREA,BUILD_YEAR,CBD_DIST,NEAREST_STN,NEAREST_STN_DIST,DATE_SOLD,POSTCODE,LATITUDE,LONGITUDE,NEAREST_SCH,NEAREST_SCH_DIST,NEAREST_SCH_RANK
0,1 Acorn Place,South Lake,565000,4,2,2.0,600,160,2003.0,18300,Cockburn Central Station,1800,2018-09,6164,-32.115900,115.842450,LAKELAND SENIOR HIGH SCHOOL,0.828339,NaN
1,1 Addis Way,Wandi,365000,3,2,2.0,351,139,2013.0,26900,Kwinana Station,4900,2019-02,6167,-32.193470,115.859554,ATWELL COLLEGE,5.524324,129.0
2,1 Ainsley Court,Camillo,287000,3,1,1.0,719,86,1979.0,22600,Challis Station,1900,2015-06,6111,-32.120578,115.993579,KELMSCOTT SENIOR HIGH SCHOOL,1.649178,113.0
3,1 Albert Street,Bellevue,255000,2,1,2.0,651,59,1953.0,17900,Midland Station,3600,2018-07,6056,-31.900547,116.038009,SWAN VIEW SENIOR HIGH SCHOOL,1.571401,NaN
4,1 Aman Place,Lockridge,325000,4,1,2.0,466,131,1998.0,11200,Bassendean Station,2000,2016-11,6054,-31.885790,115.947780,KIARA COLLEGE,1.514922,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33651,9C Gold Street,South Fremantle,1040000,4,3,2.0,292,245,2013.0,16100,Fremantle Station,1500,2016-03,6162,-32.064580,115.751820,CHRISTIAN BROTHERS' COLLEGE,1.430350,49.0
33652,9C Pycombe Way,Westminster,410000,3,2,2.0,228,114,NaN,9600,Stirling Station,4600,2017-02,6061,-31.867055,115.841403,JOHN SEPTIMUS ROE ANGLICAN COMMUNITY SCHOOL,1.679644,35.0
33653,9D Pycombe Way,Westminster,427000,3,2,2.0,261,112,NaN,9600,Stirling Station,4600,2017-02,6061,-31.866890,115.841418,JOHN SEPTIMUS ROE ANGLICAN COMMUNITY SCHOOL,1.669159,35.0
33654,9D Shalford Way,Girrawheen,295000,3,1,2.0,457,85,1974.0,12600,Warwick Station,4400,2016-10,6064,-31.839680,115.842410,GIRRAWHEEN SENIOR HIGH SCHOOL,0.358494,NaN


In [6]:
#Limit dataframe to variables of interest
clean_perth = perth_data[['SUBURB',
                          'PRICE',
                          'BEDROOMS',
                          'BATHROOMS',
                          'DATE_SOLD',
                          'POSTCODE']]
clean_perth = clean_perth.rename(columns={'DATE_SOLD': 'Date'})
#Add unique ID and add City for later merging with Sydney data
clean_perth.insert(0, 'ID', range(1, 1 + len(clean_perth)))
clean_perth['CITY'] = "Perth"
#clean_perth

In [7]:
#merge Perth data with Commodities data 
Perth_commodities_df = pd.merge(clean_data, clean_perth, on="Date",how="inner")
#Perth_commodities_df

In [8]:
#save data to perth_clean.csv
Perth_commodities_df.to_csv('Outputs/perth_housing.csv')

In [9]:
#Create Perth data frame with Iron ore prices
perth_iron_df = pd.merge(iron_data, clean_perth, on="Date",how="inner")
perth_iron_df.to_csv('Outputs/perth_iron_housing.csv')


In [10]:
# Read SydneyHousePrices.csv File and store into Pandas data frame
Sydney_to_load = "Resources/SydneyHousePrices.csv"
Sydney_data = pd.read_csv(Sydney_to_load)

In [11]:
#Add City indicator for when merged with Perth data
Sydney_data["CITY"] = "Sydney"
#sanity check to see earliest date
Sydney_data.sort_values(by=['Date'])


,Date,Id,suburb,postalCode,sellPrice,bed,bath,car,propType,CITY
92372,2000-12-01,92373,Bondi Beach,2026,3565000,10.0,5,3.0,house,Sydney
147100,2001-02-02,147101,Padstow,2211,960000,4.0,2,2.0,house,Sydney
20553,2001-03-13,20554,Allambie Heights,2100,896000,4.0,2,2.0,house,Sydney
75402,2001-04-06,75403,Cammeray,2062,463000,2.0,1,2.0,townhouse,Sydney
32743,2001-09-20,32744,Cremorne,2090,372000,2.0,1,1.0,house,Sydney
...,...,...,...,...,...,...,...,...,...,...
39731,2019-07-06,39732,Collaroy Plateau,2097,5350000,5.0,5,5.0,house,Sydney
165955,2019-07-06,165956,Annandale,2038,1955000,3.0,2,NaN,house,Sydney
149351,2019-07-06,149352,South Penrith,2750,555000,3.0,1,NaN,house,Sydney
67691,2019-07-06,67692,West Pennant Hills,2125,1682000,4.0,3,3.0,house,Sydney


In [12]:
#Change the Date from string to datetime then convert to desired format 
Sydney_data['Date'] = pd.to_datetime(Sydney_data['Date'], errors='coerce').dt.to_period('M')

Sydney_data

,Date,Id,suburb,postalCode,sellPrice,bed,bath,car,propType,CITY
0,2019-06,1,Avalon Beach,2107,1210000,4.0,2,2.0,house,Sydney
1,2019-06,2,Avalon Beach,2107,2250000,4.0,3,4.0,house,Sydney
2,2019-06,3,Whale Beach,2107,2920000,3.0,3,2.0,house,Sydney
3,2019-05,4,Avalon Beach,2107,1530000,3.0,1,2.0,house,Sydney
4,2019-05,5,Whale Beach,2107,8000000,5.0,4,4.0,house,Sydney
...,...,...,...,...,...,...,...,...,...,...
199499,2014-06,199500,Illawong,2234,1900000,5.0,3,7.0,house,Sydney
199500,2014-05,199501,Illawong,2234,980000,4.0,3,2.0,house,Sydney
199501,2014-04,199502,Alfords Point,2234,850000,4.0,2,2.0,house,Sydney
199502,2013-09,199503,Illawong,2234,640000,3.0,2,2.0,townhouse,Sydney


In [13]:
#Rename columns matching Perth data to combine later
clean_Sydney = Sydney_data.rename(columns={'Id':'ID',
                          'suburb':'SUBURB',
                          'sellPrice':'PRICE',
                          'bed':'BEDROOMS',
                          'bath':'BATHROOMS',
                          'postalCode':'POSTCODE'})
clean_Sydney

,Date,ID,SUBURB,POSTCODE,PRICE,BEDROOMS,BATHROOMS,car,propType,CITY
0,2019-06,1,Avalon Beach,2107,1210000,4.0,2,2.0,house,Sydney
1,2019-06,2,Avalon Beach,2107,2250000,4.0,3,4.0,house,Sydney
2,2019-06,3,Whale Beach,2107,2920000,3.0,3,2.0,house,Sydney
3,2019-05,4,Avalon Beach,2107,1530000,3.0,1,2.0,house,Sydney
4,2019-05,5,Whale Beach,2107,8000000,5.0,4,4.0,house,Sydney
...,...,...,...,...,...,...,...,...,...,...
199499,2014-06,199500,Illawong,2234,1900000,5.0,3,7.0,house,Sydney
199500,2014-05,199501,Illawong,2234,980000,4.0,3,2.0,house,Sydney
199501,2014-04,199502,Alfords Point,2234,850000,4.0,2,2.0,house,Sydney
199502,2013-09,199503,Illawong,2234,640000,3.0,2,2.0,townhouse,Sydney


In [14]:
#merge commodity prices with Sydney data
sydney_commodities_df = pd.merge(clean_data, clean_Sydney, on="Date",how="inner")
#sydney_commodities_df
#save data to sydney_clean.csv
sydney_commodities_df.to_csv('Outputs/sydney_housing.csv')

In [ ]:
#Create Sydney data frame with Iron ore prices
sydney_iron_df = pd.merge(iron_data, clean_Sydney, on="Date",how="inner")
sydney_iron_df.to_csv('Outputs/sydney_iron_housing.csv')
sydney_iron_df

In [ ]:
#combine Perth and Sydney data into one DataFrame
complete_city_df = pd.concat([clean_Sydney, clean_perth], ignore_index=True)
#limit variables to match eachother
complete_city_df = complete_city_df[[
                          'SUBURB',
                          'PRICE',
                          'BEDROOMS',
                          'BATHROOMS',
                          'Date',
                          'POSTCODE',
                           'CITY']]

#merge commodity prices with city data and add unique ID for each sale
complete_df = pd.merge(clean_data, complete_city_df, on="Date",how="inner")
complete_df.insert(0, 'ID', range(1, 1 + len(complete_df)))
complete_df

In [ ]:
#save data to complete_house_commodities.csv
complete_df.to_csv('Outputs/complete_house_commodities.csv')

In [ ]:
iron_city_df = pd.concat([perth_iron_df, sydney_iron_df], ignore_index=True)
#limit variables to match eachother
iron_city_df = iron_city_df[['Date',
                             'Iron_Price',
                             'SUBURB',
                             'PRICE',
                             'BEDROOMS',
                             'BATHROOMS',
                             'POSTCODE',
                             'CITY']]
iron_city_df = iron_city_df[iron_city_df['Date'] >= '2007-01']
iron_city_df
iron_city_df.to_csv('Outputs/complete_house_iron.csv')


In [ ]:
iron_city_df['BEDROOMS'].value_counts()
options = [2.0,3.0,4.0,5.0]

In [ ]:
blank_df= pd.DataFrame(columns=['Date',
                             'Iron_Price',
                             'SUBURB',
                             'PRICE',
                             'BEDROOMS',
                             'BATHROOMS',
                             'POSTCODE',
                             'CITY'])
outlier_df = blank_df
# Create empty list to fill with tumor vol data (for plotting)

# Calculate the IQR and quantitatively determine if there are any potential outliers. 
for x in options:
    
    
    # Locate the rows which contain mice on each drug and get the tumor volumes
    quartiles = iron_city_df['BEDROOMS'][iron_city_df['BEDROOMS']== x].quantile([0.25,0.5,0.75])
    
    lowq  = quartiles[0.25]
    highq = quartiles[0.75]
    in_quart_r = highq - lowq
    lower_b = (lowq - (1.5*in_quart_r))
    upper_b = (highq + (1.5*in_quart_r))
    
    single_drug_df = iron_city_df[iron_city_df['BEDROOMS'] == x].reset_index(drop=True)

    
    
    
print("potential outliers:")
outlier_df.head()


In [ ]:
outlier_df

In [ ]:
data = {}

for x in options:
    data[x] = iron_city_df[iron_city_df['BEDROOMS'] == x].reset_index(drop=True)

fig1, ax1 = plt.subplots()
ax1.set_title('house sale price across bedroom types')
dt = [data[2.0]['PRICE'],
      data[3.0]['PRICE'],
      data[4.0]['PRICE'],
      data[5.0]['PRICE']]
ax1.boxplot(dt)

plt.xlabel('Bedrooms')
plt.ylabel('Price')
plt.xticks([1,2,3,4],options)

plt.show()

In [ ]:
fig1, ax1 = plt.subplots()
ax1.set_title('house sale price across bedroom types')

In [ ]:
#create data drame of 3 bedroom houses
three_bed_df = iron_city_df[iron_city_df['BEDROOMS']==3]
three_bed_df.to_csv('Outputs/three_bed_housing.csv')

In [ ]:
fig1, ax1 = plt.subplots()
ax1.set_title('house sale price across bedroom types')
ax1.boxplot(three_bed_df["PRICE"])

plt.xlabel('Bedrooms')
plt.ylabel('Price')



In [ ]:
#create data drame of 4 bedroom houses
four_bed_df = iron_city_df[iron_city_df['BEDROOMS']==4]
four_bed_df.to_csv('Outputs/four_bed_housing.csv')

In [ ]:
#create data drame of 4 bedroom 2 bathroom houses
four_bed_2bath_df = four_bed_df[four_bed_df['BATHROOMS']==2]
four_bed_2bath_df.to_csv('Outputs/four_bed_two_bath_housing.csv')

fig1, ax1 = plt.subplots()
ax1.set_title('house sale price across bedroom types')
ax1.boxplot(four_bed_2bath_df["PRICE"])

plt.xlabel('Bedrooms')
plt.ylabel('Price')


In [ ]:
quartiles = four_bed_2bath_df['PRICE'].quantile([0.25,0.5,0.75])
    
lowq  = quartiles[0.25]
highq = quartiles[0.75]
in_quart_r = highq - lowq
lower_b = (lowq - (1.5*in_quart_r))
upper_b = (highq + (1.5*in_quart_r))

trim_4b_2bath_df = four_bed_2bath_df[four_bed_2bath_df["PRICE"] <= upper_b]


In [ ]:
trim_4b_2bath_df.to_csv('Outputs/four_bed_two_bath_housing.csv')

In [ ]:
print(lowq,highq,in_quart_r,lower_b,upper_b)

In [ ]:
upper_b

In [ ]:
fig1, ax1 = plt.subplots()
ax1.set_title('Four Bedroom Two bathroom house price')
ax1.boxplot(trim_4b_2bath_df["PRICE"])

plt.xlabel('4 bedroom')
plt.ylabel('Price')

In [ ]:
y2007 = trim_4b_2bath_df[(trim_4b_2bath_df["Date"]>= '2007-01') & (trim_4b_2bath_df["Date"] <= '2007-12')]['PRICE']
y2008 = trim_4b_2bath_df[(trim_4b_2bath_df["Date"]>= '2008-01') & (trim_4b_2bath_df["Date"] <= '2008-12')]['PRICE']
y2009 = trim_4b_2bath_df[(trim_4b_2bath_df["Date"]>= '2009-01') & (trim_4b_2bath_df["Date"] <= '2009-12')]['PRICE']
y2010 = trim_4b_2bath_df[(trim_4b_2bath_df["Date"]>= '2010-01') & (trim_4b_2bath_df["Date"] <= '2010-12')]['PRICE']
y2011 = trim_4b_2bath_df[(trim_4b_2bath_df["Date"]>= '2011-01') & (trim_4b_2bath_df["Date"] <= '2011-12')]['PRICE']
y2012 = trim_4b_2bath_df[(trim_4b_2bath_df["Date"]>= '2012-01') & (trim_4b_2bath_df["Date"] <= '2012-12')]['PRICE']
y2013 = trim_4b_2bath_df[(trim_4b_2bath_df["Date"]>= '2013-01') & (trim_4b_2bath_df["Date"] <= '2013-12')]['PRICE']
y2014 = trim_4b_2bath_df[(trim_4b_2bath_df["Date"]>= '2014-01') & (trim_4b_2bath_df["Date"] <= '2014-12')]['PRICE']
y2015 = trim_4b_2bath_df[(trim_4b_2bath_df["Date"]>= '2015-01') & (trim_4b_2bath_df["Date"] <= '2015-12')]['PRICE']
y2016 = trim_4b_2bath_df[(trim_4b_2bath_df["Date"]>= '2016-01') & (trim_4b_2bath_df["Date"] <= '2016-12')]['PRICE']
y2017 = trim_4b_2bath_df[(trim_4b_2bath_df["Date"]>= '2017-01') & (trim_4b_2bath_df["Date"] <= '2017-12')]['PRICE']
y2018 = trim_4b_2bath_df[(trim_4b_2bath_df["Date"]>= '2018-01') & (trim_4b_2bath_df["Date"] <= '2018-12')]['PRICE']
y2019 = trim_4b_2bath_df[(trim_4b_2bath_df["Date"]>= '2019-01') & (trim_4b_2bath_df["Date"] <= '2019-12')]['PRICE']

In [ ]:
data = [y2007,
        y2008,
        y2009,
        y2010,
        y2011,
        y2012,
        y2013,
        y2014,
        y2015,
        y2016,
        y2017,
        y2018,
        y2019] 

In [ ]:
#trim_4b_2bath_df["Date"] = pd.to_datetime(trim_4b_2bath_df["Date"]).dt.to_period('M')

fig1, ax1 = plt.subplots()
ax1.set_title('Four Bedroom Two bathroom house price')
#                 clean_data[(clean_data['Date'] >= '2007-01') & (clean_data['Date'] <= '2019-06')]
ax1.boxplot(data)

plt.xlabel('4 bedroom')
plt.ylabel('Price')
#plt.xticks(np.arange(2007,2020,1))

In [ ]:
fig1, ax1 = plt.subplots()
ax1.set_title('house sale price across bedroom types')
dt = [trim_4b_2bath_df[[]]["PRICE"],
      data[3.0]['PRICE'],
      data[4.0]['PRICE'],
      data[5.0]['PRICE']]
ax1.boxplot(dt)

plt.xlabel('Years')
plt.ylabel('Price')
#plt.xticks([1,2,3,4],options)

plt.show()